In [5]:
import boto3
import sagemaker
from sagemaker.amazon.amazon_estimator import image_uris
container = image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version="1")

In [9]:
import re
from sagemaker import get_execution_role

role = get_execution_role() 

In [6]:
bucket="dhana-sagemaker-workshop"
prefix ="dhana-s3-mle"

In [11]:
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

In [13]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

print(xgb.latest_training_job.name)

INFO:sagemaker:Creating training-job with name: xgboost-2024-01-04-19-42-16-067


2024-01-04 19:42:16 Starting - Starting the training job...
2024-01-04 19:42:39 Starting - Preparing the instances for training.........
2024-01-04 19:44:04 Downloading - Downloading input data...
2024-01-04 19:44:39 Downloading - Downloading the training image......
2024-01-04 19:45:45 Training - Training image download completed. Training in progress.
2024-01-04 19:45:45 Uploading - Uploading generated training modelArguments: train
[2024-01-04:19:45:35:INFO] Running standalone xgboost training.
[2024-01-04:19:45:35:INFO] File size need to be processed in the node: 4.35mb. Available memory size in the node: 8546.58mb
[2024-01-04:19:45:35:INFO] Determined delimiter of CSV input is ','
[19:45:35] S3DistributionType set as FullyReplicated
[19:45:35] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-01-04:19:45:35:INFO] Determined delimiter of CSV input is ','
[19:45:35] S3DistributionType set as FullyReplicated
[19:45:3